<font size=7>Reliability of the salience features and object categories<font> 

<font size=3>Finally, we want to look at the split-half reliability of the salience features and the object categories in our data set. The **extract_fix_split_half()** function provides a dataframe with two columns for each saliency feature and object category (color, intensity, orientation, text, face, taste, motion, touched). These correspond to values from even and odd images. Each row contains the data of one subject.

* <font size=3>Calculate the split-half reliability for all saliency features and object categories. How do you assess the reliabilities?<font>

In [9]:
import os, glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.io
import scipy.stats
from scipy.ndimage import gaussian_filter
import ipywidgets as widgets
from IPython.display import display, HTML
from PIL import Image
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import cv2
import ITTISaliencyLib
import warnings

warnings.filterwarnings('ignore')

IMAGE_FOLDER = 'images_task_F'
image_files = [f for f in os.listdir(IMAGE_FOLDER) if f.endswith('.jpg')]
RESULTS_FOLDER_FIX = 'data_task_C/data_fix/'
RESULTS_FOLDER_SACC = 'data_task_C/data_sacc/'
#SALIENCY_MAP_DICT = 'data_task_F/saliency_map_dict.npy'
OSIE_FILE = 'data_task_F/attrs_OSIE_40.mat'
#SUBJECT_LIST = [sub[:2] for sub in os.listdir(RESULTS_FOLDER_FIX)if sub != '.DS_Store']
#print(SUBJECT_LIST)

***The cell below might take a long time to execute:***

In [10]:
def make_saliency_map_dict():
    #images_path = "/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/OSIE_40/images"
    #image_list = []

    #os.chdir(images_path)
    #os.chdir(IMAGE_FOLDER)
    #for file in glob.glob("*.jpg"):
    #    image_list.append(file)

    saliency_map_dict = {}

    #for image_name in image_list:
    for image_name in image_files:
        img = cv2.imread(os.path.join(IMAGE_FOLDER,image_name))
        saliency_map_dict[image_name] = {}

        # image_num = image_name[0:4]
        # initialize
        # imgsize = img.shape
        # img_width = imgsize[1]
        # img_height = imgsize[0]
        # sm = pySaliencyMap.pySaliencyMap(img_width, img_height)
        # computation of saliency maps
        saliency_map = ITTISaliencyLib.get_spatial_saliency_map(img)
        saliency_map_intensity = ITTISaliencyLib.get_spatial_saliency_map_I(img)
        saliency_map_color = ITTISaliencyLib.get_spatial_saliency_map_C(img)
        saliency_map_orientation = ITTISaliencyLib.get_spatial_saliency_map_O(img)

        list_of_maps = [
            ("saliency_map", saliency_map),
            ("saliency_map_intensity", saliency_map_intensity),
            ("saliency_map_color", saliency_map_color),
            ("saliency_map_orientation", saliency_map_orientation),
        ]

        saliency_map_dict[image_name].update(list_of_maps)

    return saliency_map_dict

SALIENCY_MAP_DICT = make_saliency_map_dict()

In [16]:
def extract_semantic_saliency_fix(subject_num, group_input):
    # Add necessary path
    #fix_data_path = (
    #    #"/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/data/derivatives/"
    #    RESULTS_FOLDER_FIX
    #    + subject_num
    #    + "_fix_data.csv"
    #)
    if group_input == "A":
        fix_data_path = (
        RESULTS_FOLDER_FIX 
            +"group_A/"
            + subject_num
            + "_fix_data.csv"
    )
    if group_input== "B":
        fix_data_path = (
        RESULTS_FOLDER_FIX 
            +"group_B/"
            + subject_num
            + "_fix_data.csv"
    )

    OSIE_data_path = OSIE_FILE#"/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/OSIE_40/attrs_OSIE_40.mat"

    #saliency_map_path = SALIENCY_MAP_DICT#"/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/OSIE_40/saliency_map_dict.npy"

    # Load the MATLAB .mat file containing the data
    fix_data = pd.read_csv(fix_data_path, sep=",")
    # mat_data = loadmat(OSIE_data_path)
    mat_dict = loadmat(OSIE_data_path, simplify_cells=True)
    attrs_OSIE_40 = mat_dict["attrs_OSIE_40"]
    #saliency_map_dict = np.load(
    #    saliency_map_path,
    #    allow_pickle="TRUE",
    #).item()
    saliency_map_dict = SALIENCY_MAP_DICT
    

    # print(saliency_map_dict)
    # df = pd.read_table("/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/pilot_res/pp_scan_2024_03_21_16_08.asc")
    # ascii_grid = np.loadtxt("/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/pilot_res/pp_scan_2024_03_21_16_08.asc", skiprows=100)

    # Constants and parameters
    dims = [0, 1, 5, 7, 10]
    # Header = ["Text", "Face", "Taste", "Motion", "Touched"]

    im_width, im_height = 800, 600

    im_width_real, im_height_real = 1146, 876  # real size
    exp_scale_width = im_width_real / im_width  # 1.4325
    exp_scale_height = im_height_real / im_height  # 1.46
    screen_width, screen_height = 1920, 1080
    # screen_x_cm, screen_y_cm, vp_dist_cm = 51.5, 29.0, 60
    # sample_freq = 1000
    # XPIX2DEG = np.arctan((screen_x_cm / screen_width) / vp_dist_cm) * (180 / np.pi)
    # YPIX2DEG = np.arctan((screen_y_cm / screen_height) / vp_dist_cm) * (180 / np.pi)
    # folder_name = f"e1v{subject_num}b1"
    fixations = {}

    # im_num = '1001.jpg'
    # block_num = 1

    BLOCK_NUMS = 40
    for block_num in range(0, BLOCK_NUMS):
        # block_num = 1
        image_name = attrs_OSIE_40[block_num]["img"]
        # objs = attrs_OSIE_40[block_num]["objs"]

        fix_data_block = fix_data[fix_data["img"] == image_name]
        Cfix = len(fix_data_block)  # fixation count
        Dfix = list(fix_data_block["dur"])  # fixation durations
        Xfix = list(
            ((fix_data_block["axp"] - (screen_width / 2)) / exp_scale_width) + (im_width / 2)
        )
        # Xfix = Xfix - (screen_width/2) #/ exp_scale_width)
        # Xfix = Xfix + (im_width/2)
        # Xfix_norm = Xfix/exp_scale_width # X coord
        Yfix = list(
            ((fix_data_block["ayp"] - (screen_height / 2)) / exp_scale_height) + (im_height / 2)
        )  # / exp_scale_height)  # Y coord
        Order = list(fix_data_block["fix_order"])
        # for im_num in pd.unique(fix_data['img']):

        # Analyze semantic content
        sem = np.zeros((Cfix, len(dims)))

        for d in range(len(dims)):
            map_sum = np.zeros((im_height, im_width))
            # map_ori = []
            for obj in range(len(attrs_OSIE_40[block_num]["objs"])):
                # obj = 0
                if attrs_OSIE_40[block_num]["objs"][obj]["features"][dims[d]] > 0:
                    # print(attrs_OSIE_40[block_num-1]['objs'][obj]['features'][dims[d]])
                    map_ori = attrs_OSIE_40[block_num]["objs"][obj]["map"]
                    # print(map_ori)
                    map_sum += map_ori
                    # print(map_sum)
            for f in range(Cfix):
                sem[f, d] = 0
                if 0 < round(Xfix[f]) <= im_width and 0 < round(Yfix[f]) <= im_height:
                    sem[f, d] = map_sum[int(round(Yfix[f])), int(round(Xfix[f]))] * Dfix[f]

        sem_df = pd.DataFrame(sem)

        # Analyze saliency content
        sal = np.zeros((Cfix, len(saliency_map_dict[image_name])))
        list_of_sal_maps = list(saliency_map_dict[image_name].keys())

        for i, sal_map in enumerate(list_of_sal_maps):
            # print(i, sal_map)
            sal_map_ori = saliency_map_dict[image_name][sal_map]

            for f in range(Cfix):
                sal[f, i] = 0
                if 0 < round(Xfix[f]) <= im_width and 0 < round(Yfix[f]) <= im_height:
                    # sal[f, i] = sal_map_ori[int(round(Yfix[f])), int(round(Xfix[f]))] * Dfix[f]
                    sal[f, i] = (
                        np.sum(sal_map_ori < sal_map_ori[int(round(Yfix[f])), int(round(Xfix[f]))])
                        / sal_map_ori.size
                    )  # * Dfix[f]

        sal_df = pd.DataFrame(sal, columns=list_of_sal_maps)

        fixations[image_name] = {
            "X": Xfix,
            "Y": Yfix,
            # "Onset": Tfix,
            "Duration": Dfix,
            # "Amplitude": Asac,
            "Sem": sem_df,
            "Sal": sal_df,
            "Order": Order,
        }

    return fixations



def extract_fix_split_half(group_input):
    # fixations_odd = {}
    # fixations_even = {}
    if group_input=='A':
        SUBJECT_LIST = [sub[:2] for sub in os.listdir(RESULTS_FOLDER_FIX+"group_A/")if sub != '.DS_Store']
        
    else:
        SUBJECT_LIST = [sub[:2] for sub in os.listdir(RESULTS_FOLDER_FIX+"group_B/")if sub != '.DS_Store']
    
    fix_dataframe_group = pd.DataFrame()
    #subject_list = ["zt", "pp", "jp"]
    
    #for subject_num in subject_list:
    for subject_num in SUBJECT_LIST:
        # subject_num='zt'
        # fixations = extract_semantic_fix(subject_num)
        fixations = extract_semantic_saliency_fix(subject_num, group_input)
        # Constants and parameters
        dims = [0, 1, 5, 7, 10]
        header_sem1 = ["Text1", "Face1", "Taste1", "Motion1", "Touched1"]
        header_sal1 = ["Saliency1", "Intensity1", "Color1", "Orientation1"]
        header_sem2 = ["Text2", "Face2", "Taste2", "Motion2", "Touched2"]
        header_sal2 = ["Saliency2", "Intensity2", "Color2", "Orientation2"]
        header_tot = header_sem1 + header_sem2 + header_sal1 + header_sal2

        first_fix = []
        all_fix_sem_1 = pd.DataFrame()
        all_fix_sal_1 = pd.DataFrame()
        all_fix_sem_2 = pd.DataFrame()
        all_fix_sal_2 = pd.DataFrame()
        # d_fix = []
        all_dur_1 = []  # pd.DataFrame()
        all_dur_2 = []
        fixation_keys = list(fixations.keys())

        for i, key in enumerate(fixation_keys):
            # key = '1001.jpg'
            if fixations[key] is not None:
                # sem
                df_sem = pd.DataFrame(fixations[key]["Sem"])
                df_sal = fixations[key]["Sal"]
                df_dur = fixations[key]["Duration"]

                if i % 2 == 0:
                    all_fix_sem_2 = pd.concat([all_fix_sem_2, df_sem])
                    all_dur_2 += df_dur
                    all_fix_sal_2 = pd.concat([all_fix_sal_2, df_sal])

                else:
                    all_fix_sem_1 = pd.concat([all_fix_sem_1, df_sem])
                    all_dur_1 += df_dur
                    all_fix_sal_1 = pd.concat([all_fix_sal_1, df_sal])

        # first_prop = np.sum(np.sign(first_fix), axis=0) / len(first_fix)
        # all_prop = np.sum(all_fix, axis=0) / np.sum(d_fix)
        all_prop_sem_1 = all_fix_sem_1.sum() / sum(all_dur_1)
        all_prop_sem_2 = all_fix_sem_2.sum() / sum(all_dur_2)
        all_prop_sal_1 = all_fix_sal_1.mean()
        all_prop_sal_2 = all_fix_sal_2.mean()

        # sem_fix_data = pd.DataFrame((np.array(header), first_prop, np.array(all_prop))).T.rename(columns={0:'header', 1:'first_prop', 2:'all_prop'})
        fix_data_ind = {
            # "header": header_sem1 + header_sem2 + header_sal1 + header_sal2,
            # "first_prop": list(first_prop),
            "all_prop": list(all_prop_sem_1)
            + list(all_prop_sem_2)
            + list(all_prop_sal_1)
            + list(all_prop_sal_2),
        }
        fix_dataframe_ind = pd.DataFrame.from_dict(
            fix_data_ind, orient="index", columns=header_tot
        )  # , columns=["header", "all_prop"]).T
        fix_dataframe_ind["sub_num"] = subject_num
        # fix_dataframe_ind(columns="header")
        fix_dataframe_group = pd.concat([fix_dataframe_group, fix_dataframe_ind])
        
    return fix_dataframe_group



<font size=3>To give a name (e.g. 'df') to and visualize the dataframe use the **extract_fix_split_half()** function. 
Note that this function has a parameter, your group ('A' or 'B', as character):

```python
df = extract_fix_split_half('group')
display(df)
```

<font size=3>To easily access two columns and give them each a name:
```python
Feature1 = df['Feature1']
Feature2 = df['Feature2']
print(Feature1)
print(Feature2)
```

<font size=3> Try yourself:

<font size=3>You can calculate and print the Pearson correlation coefficient and the corresponding p-value as follows:
```python
R_feat, p_feat = scipy.stats.pearsonr(Feature1, Feature2)
print(R_feat, p_feat)
```


You can use the cells like a calculator, doing simple operations like:
- additions:
```python
sum = a + b
```
- substractions:
```python
difference = a - b
```
- multiplications:
```python
product = a * b
```
- divisions:
```python
quotient = a / b
```
- or a combination of any of these...